In [2]:
# ECE 475 Assignment 3: Model Assessment and Selection
# Re-implement the example in section 7.10.2 using any simple, out of the box classifier (like K nearest neighbors from sci-kit). 
# Reproduce the results for the incorrect and correct way of doing cross-validation.

# Sophie, Rebecca, and Marc

import numpy as np
import pandas as pd
import random
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest


In [33]:
# Create Features
# Standard Gaussian Predictors 5000 predictors x 50 samples
mu, sigma = 0, 0.1 # mean and standard deviation
N = 50 # number of samples
p = 5000 # number of predictors 

s = np.random.normal(mu, sigma, [N,p])
# Create Labels
z = np.zeros(25)
o = np.ones(25)
labels = np.append(z, o)
random.shuffle(labels)

def split_data(s):

  percent_correct_incorrect = np.zeros(50)
  percent_correct_correct = np.zeros(50)

  # Split in K cross-validation folds randomly 
  K = 5
  fold_sze = int(N/K)

  # Get the indices
  index = [0] * (K+1)
  for i in range(1,K+1):
    index[i] = int(fold_sze)*(i)

  all_folds = [[0] * 1] * (K+1)
  all_fold_labels = [[0] * 1] * (K+1)
  # Create K folds
  start = 1
  for k in range(1,K+1):
    end = index[k]
    fold = s[start-1:end]
    fold_label = labels[start-1:end]

    all_folds[k] = fold
    all_fold_labels[k] = fold_label
    start = end + 1

    # Display Folds 
    df = pd.DataFrame(data=fold)

  all_folds = np.delete(all_folds,0,axis = 0)
  all_fold_labels = np.delete(all_fold_labels,0,axis = 0)

  df = pd.DataFrame(data=all_folds)

  return all_folds, all_fold_labels

In [55]:
for n in range(0,50):

  # Incorrect cross validation

  model_features = SelectKBest(k=100)
  best_features = model_features.fit_transform(s,labels)

  all_folds, all_fold_labels = split_data(best_features)



  correct = 0

  for k in range(0,K):
    training_data = np.concatenate([all_folds[i] for i in range(5) if (i != k)]) # This line was given to us by Andrew Lorber
    training_labels = np.concatenate([all_fold_labels[i] for i in range(5) if(i != k)])
    #training_data_subset = np.concatenate([best_features[i] for i in range(5) if (i != k)]) # This line was given to us by Andrew Lorber
    #print(training_data_subset)
    #training_labels_subset = np.concatenate([all_fold_labels[i] for i in range(5) if (i != k)])
    
    testing_data = all_folds[k]
    testing_labels = all_fold_labels[k]
    
    # Display training data (all samples but kth fold)
    df = pd.DataFrame(data=training_data)

    neigh = KNeighborsClassifier(n_neighbors=1)
    neigh.fit(training_data, training_labels)

    #predicted_labels = neigh.predict(model_features.transform(testing_data))
    predicted_labels = neigh.predict((testing_data))

    for i in range(0,len(predicted_labels)):
      if predicted_labels[i] == testing_labels[i]:
        correct = correct + 1
  percent_correct_incorrect[n] = correct/(K*len(testing_labels))


  # K-fold Cross Validation
  correct = 0

  for k in range(0,K):
    all_folds, all_fold_labels = split_data(s)
    training_data = np.concatenate([all_folds[i] for i in range(5) if (i != k)]) # This line was given to us by Andrew Lorber
    training_labels = np.concatenate([all_fold_labels[i] for i in range(5) if (i != k)])
    testing_data = all_folds[k]
    testing_labels = all_fold_labels[k]
    # Display training data (all samples but kth fold)
    df = pd.DataFrame(data=training_data)

    model_features = SelectKBest(k=100)
    best_features = model_features.fit_transform(training_data,training_labels)

    neigh = KNeighborsClassifier(n_neighbors=1)
    neigh.fit(best_features,training_labels)

    predicted_labels = neigh.predict(model_features.transform(testing_data))
    for i in range(0,len(predicted_labels)):
      if predicted_labels[i] == testing_labels[i]:
        correct = correct + 1
        
  percent_correct_correct[n] = correct/(K*len(testing_labels))


print('Incorrect Method Percent Correct =', np.mean(percent_correct_incorrect) * 100)
print('Correct Method Percent Correct =', np.mean(percent_correct_correct) * 100)




Incorrect Method Percent Correct = 98.0
Correct Method Percent Correct = 50.0


# Model Assessment and Selection

In this project cross validation is done incorrectly and then correctly. Using the incorrect method, the 100 best predictors are chosen based on all of the samples. The model then overfits and has a very high percent correct, which came out to 98% in this case. When using the correct method for cross validation, each of k folds was used as the testing data while the other k-1 folds were used as the training data. The 100 best predictors are chosen on each set of training data and then trained on the corresponding testing data. The model predicts 50% correct.